<a href="https://colab.research.google.com/github/maxfomichev-tech/news-parser/blob/main/news_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# ╔══════════════════════════════════════════════════════════════════════╗
# ║           TELEGRAM NEWS BOT для Google Colab                         ║
# ╚══════════════════════════════════════════════════════════════════════╝

# ┌─────────────────────────────────────────────────────────────────────┐
# │ ЯЧЕЙКА 1: Установка зависимостей                                    │
# └─────────────────────────────────────────────────────────────────────┘

!pip install -q python-telegram-bot==20.7 aiohttp==3.9.1 beautifulsoup4==4.12.2 \
    python-dotenv==1.0.0 lxml==4.9.3 tenacity==8.2.3 html2text==2020.1.16

print("✅ Зависимости установлены")

✅ Зависимости установлены


In [16]:
from google.colab import userdata
import os

# Читаем секреты
os.environ['TELEGRAM_BOT_TOKEN'] = userdata.get('TELEGRAM_BOT_TOKEN')
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')  # Новый ключ!

print("✅ Токены загружены")

if not os.environ['TELEGRAM_BOT_TOKEN']:
    print("❌ TELEGRAM_BOT_TOKEN не найден!")
else:
    print("✅ TELEGRAM_BOT_TOKEN на месте")

if not os.environ['GROQ_API_KEY']:
    print("❌ GROQ_API_KEY не найден!")
    print("   Получите ключ: https://console.groq.com/keys")
else:
    print("✅ GROQ_API_KEY на месте")

# ═══════════════════════════════════════════════════════════════════
# CONFIG для Groq
# ═══════════════════════════════════════════════════════════════════

config_code = '''
import os

class Config:
    TELEGRAM_TOKEN = os.getenv('TELEGRAM_BOT_TOKEN', '')
    GROQ_API_KEY = os.getenv('GROQ_API_KEY', '')
    RSS_FEEDS_FILE = "rss_feeds.txt"
    MAX_NEWS_ITEMS = 30
    MAX_CHARS_FOR_ANALYSIS = 8000
    TG_MAX_MESSAGE_LENGTH = 4096
    COMMAND_COOLDOWN_SECONDS = 30

config = Config()
'''

rss_feeds = '''https://lenta.ru/rss/news
https://meduza.io/rss/all
https://tass.ru/rss/v2.xml'''

# utils.py — без изменений
utils_code = '''
import re
import html
import html2text
from typing import List

html_converter = html2text.HTML2Text()
html_converter.ignore_links = False
html_converter.ignore_images = True
html_converter.body_width = 0

def clean_html(raw_html: str) -> str:
    if not raw_html:
        return ""
    try:
        text = html_converter.handle(raw_html)
        text = re.sub(r'\\n{3,}', '\\n\\n', text)
        return text.strip()
    except Exception:
        clean = re.sub(r'<[^>]+>', '', raw_html)
        return html.unescape(clean).strip()

def split_message(text: str, max_length: int = 4096) -> List[str]:
    if len(text) <= max_length:
        return [text]
    parts = []
    current = ""
    for paragraph in text.split('\\n\\n'):
        if len(current) + len(paragraph) + 2 > max_length:
            if current:
                parts.append(current.strip())
            current = paragraph
        else:
            current += "\\n\\n" + paragraph if current else paragraph
    if current:
        parts.append(current.strip())
    return parts

class RateLimiter:
    def __init__(self, cooldown):
        self.cooldown = cooldown
        self.last_request = {}

    def can_proceed(self, user_id: int) -> bool:
        from datetime import datetime
        now = datetime.now()
        if user_id not in self.last_request:
            self.last_request[user_id] = now
            return True
        elapsed = (now - self.last_request[user_id]).total_seconds()
        if elapsed >= self.cooldown:
            self.last_request[user_id] = now
            return True
        return False

    def get_remaining_time(self, user_id: int) -> int:
        from datetime import datetime
        if user_id not in self.last_request:
            return 0
        elapsed = (datetime.now() - self.last_request[user_id]).total_seconds()
        return max(0, int(self.cooldown - elapsed))
'''

# rss_fetcher.py — без изменений
rss_fetcher_code = '''
import asyncio
import aiohttp
import xml.etree.ElementTree as ET
from dataclasses import dataclass
from datetime import datetime
from typing import List, Optional
from utils import clean_html
from config import config

@dataclass
class NewsItem:
    title: str
    summary: str
    link: str
    source: str
    published: Optional[datetime] = None
    raw_date: str = ""

class RSSFetcher:
    def __init__(self, timeout: int = 30):
        self.timeout = aiohttp.ClientTimeout(total=timeout)
        self.session = None

    async def __aenter__(self):
        self.session = aiohttp.ClientSession(timeout=self.timeout)
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        if self.session:
            await self.session.close()

    async def fetch_feed(self, url: str) -> List[NewsItem]:
        if not self.session:
            raise RuntimeError("Use async with")

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept': 'application/rss+xml, application/xml, text/xml, */*',
            'Accept-Language': 'ru-RU,ru;q=0.9',
        }

        try:
            async with self.session.get(url, headers=headers) as response:
                if response.status != 200:
                    print(f"⚠️ {url}: HTTP {response.status}")
                    return []
                content = await response.text()
                return self._parse_xml(content, url)
        except Exception as e:
            print(f"❌ Error fetching {url}: {e}")
            return []

    def _parse_xml(self, content: str, source_url: str) -> List[NewsItem]:
        items = []
        try:
            root = ET.fromstring(content)
        except ET.ParseError as e:
            print(f"XML parse error: {e}")
            return []

        ns = {'atom': 'http://www.w3.org/2005/Atom'}
        channel = root.find('.//channel')
        feed = root.find('.//{http://www.w3.org/2005/Atom}feed')

        source_name = source_url.split('/')[2] if '://' in source_url else "Unknown"

        if channel is not None:
            title_elem = channel.find('title')
            source_name = title_elem.text if title_elem is not None else source_name
            entries = channel.findall('item')
            date_tag, desc_tag = 'pubDate', 'description'
        elif feed is not None:
            title_elem = feed.find('atom:title', ns)
            source_name = title_elem.text if title_elem is not None else source_name
            entries = feed.findall('atom:entry', ns)
            date_tag, desc_tag = 'published', 'summary'
        else:
            entries = root.findall('.//item') or root.findall('.//entry', ns)
            date_tag, desc_tag = 'pubDate', 'description'

        for entry in entries[:10]:
            try:
                title = self._get_text(entry, 'title', ns) or "Без заголовка"
                link = self._get_link(entry, ns)
                summary = self._get_text(entry, desc_tag, ns) or ""

                items.append(NewsItem(
                    title=title.strip(),
                    summary=clean_html(summary)[:300],
                    link=link,
                    source=source_name,
                    raw_date=self._get_text(entry, date_tag, ns) or ""
                ))
            except Exception as e:
                continue

        return items

    def _get_text(self, element, tag: str, ns: dict) -> Optional[str]:
        if ns:
            elem = element.find(f'atom:{tag}', ns)
            if elem is not None and elem.text:
                return elem.text
        elem = element.find(tag)
        if elem is not None and elem.text:
            return elem.text
        return None

    def _get_link(self, element, ns: dict) -> str:
        link_elem = element.find('atom:link', ns) if ns else None
        if link_elem is not None:
            href = link_elem.get('href')
            if href:
                return href

        link_elem = element.find('link')
        if link_elem is not None and link_elem.text:
            return link_elem.text

        guid = element.find('guid')
        if guid is not None and guid.text:
            return guid.text
        return ""

    async def fetch_all(self, urls: List[str]) -> List[NewsItem]:
        tasks = [self.fetch_feed(url) for url in urls]
        results = await asyncio.gather(*tasks, return_exceptions=True)

        all_items = []
        for result in results:
            if isinstance(result, list):
                all_items.extend(result)
            elif isinstance(result, Exception):
                print(f"Task failed: {result}")

        return all_items[:config.MAX_NEWS_ITEMS]

def load_rss_urls(filename: str) -> List[str]:
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            urls = []
            for line in f:
                line = line.strip()
                if not line or line.startswith('#'):
                    continue
                if line.startswith(('http://', 'https://')):
                    urls.append(line)
            return urls
    except FileNotFoundError:
        print(f"File {filename} not found")
        return []
'''

# ═══════════════════════════════════════════════════════════════════
# AI ANALYZER для Groq (быстрый и бесплатный)
# ═══════════════════════════════════════════════════════════════════

ai_analyzer_code = '''
import aiohttp
from config import config

class AIAnalyzer:
    def __init__(self):
        self.api_key = config.GROQ_API_KEY
        self.url = "https://api.groq.com/openai/v1/chat/completions"
        self.model = "llama-3.1-8b-instant"

    async def analyze_news(self, news_text: str) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }

        prompt = f"""Ты — профессиональный аналитик новостей. Проанализируй новости и создай структурированный отчет.

📊 Основные темы (3-5 тем с кратким описанием)
🔥 Главные события (2-3 события с контекстом)
📈 Тренды (что набирает обороты)
💡 Вывод (общая оценка ситуации)

Будь объективным и лаконичным.

Новости для анализа:
{news_text[:4000]}"""

        payload = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.5,
            "max_tokens": 1000
        }

        timeout = aiohttp.ClientTimeout(total=30)  # Groq очень быстрый

        async with aiohttp.ClientSession(timeout=timeout) as session:
            async with session.post(
                self.url,
                headers=headers,
                json=payload
            ) as response:

                if response.status != 200:
                    error_text = await response.text()
                    raise Exception(f"API {response.status}: {error_text[:200]}")

                data = await response.json()
                return data['choices'][0]['message']['content'].strip()

    async def analyze_with_fallback(self, news_text: str) -> str:
        try:
            return await self.analyze_news(news_text)
        except Exception as e:
            print(f"AI Error: {e}")
            lines = [l.strip() for l in news_text.split('\\n') if l.strip() and l[0].isdigit()]
            headers = [l.split('. ', 1)[1] if '. ' in l else l for l in lines[:5]]

            return (
                "⚠️ *Анализ недоступен*\\n\\n"
                f"_Ошибка: {str(e)[:100]}_\\n\\n"
                "*Последние новости:*\\n" +
                '\\n'.join([f"• {h[:80]}..." for h in headers])
            )
'''

# Сохраняем файлы
files = {
    'config.py': config_code,
    'rss_feeds.txt': rss_feeds,
    'utils.py': utils_code,
    'rss_fetcher.py': rss_fetcher_code,
    'ai_analyzer.py': ai_analyzer_code
}

for filename, content in files.items():
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"✅ {filename}")

print("\n🎉 Готово! Запускайте ячейку 3")

✅ Токены загружены
✅ TELEGRAM_BOT_TOKEN на месте
✅ GROQ_API_KEY на месте
✅ config.py
✅ rss_feeds.txt
✅ utils.py
✅ rss_fetcher.py
✅ ai_analyzer.py

🎉 Готово! Запускайте ячейку 3


In [ ]:
import nest_asyncio
nest_asyncio.apply()

from telegram import Update
from telegram.ext import Application, CommandHandler, ContextTypes
import asyncio

# Import modules as objects to allow reloading
import config as config_module
import rss_fetcher as rss_fetcher_module
import ai_analyzer as ai_analyzer_module
import utils as utils_module

# Reload modules to ensure latest changes are picked up
import importlib
importlib.reload(config_module)
importlib.reload(rss_fetcher_module)
importlib.reload(ai_analyzer_module)
importlib.reload(utils_module)

# Now, use the reloaded objects/classes
config = config_module.config
RSSFetcher = rss_fetcher_module.RSSFetcher
load_rss_urls = rss_fetcher_module.load_rss_urls
AIAnalyzer = ai_analyzer_module.AIAnalyzer
split_message = utils_module.split_message
RateLimiter = utils_module.RateLimiter

rate_limiter = RateLimiter(config.COMMAND_COOLDOWN_SECONDS)

def prepare_news_text(news_items):
    parts = []
    total = 0
    for i, item in enumerate(news_items, 1):
        text = f"{i}. {item.title}\n"
        if item.summary:
            text += f"   {item.summary[:150]}\n"
        text += f"   [{item.source}]\n\n"
        if total + len(text) > config.MAX_CHARS_FOR_ANALYSIS:
            break
        parts.append(text)
        total += len(text)
    return "".join(parts)

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "👋 Бот для анализа новостей!\n\n"
        "/news — получить анализ текущих событий\n"
        "/status — проверить источники\n"
        "⏱️ Кулдаун между запросами: 30 сек"
    )

async def status_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    urls = load_rss_urls(config.RSS_FEEDS_FILE)
    await update.message.reply_text(f"📋 Подключено {len(urls)} RSS-каналов")

async def news_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = update.effective_user.id

    if not rate_limiter.can_proceed(user_id):
        remaining = rate_limiter.get_remaining_time(user_id)
        await update.message.reply_text(f"⏳ Подождите {remaining} секунд")
        return

    msg = await update.message.reply_text("⏳ Собираю новости из RSS...")

    try:
        urls = load_rss_urls(config.RSS_FEEDS_FILE)
        if not urls:
            await msg.edit_text("❌ Нет RSS-каналов в rss_feeds.txt")
            return

        async with RSSFetcher() as fetcher:
            news = await fetcher.fetch_all(urls)

        if not news:
            await msg.edit_text("❌ Не удалось получить новости")
            return

        await msg.edit_text(f"📥 Получено {len(news)} новостей. Анализирую...")

        news_text = prepare_news_text(news)
        analyzer = AIAnalyzer()
        analysis = await analyzer.analyze_with_fallback(news_text)

        parts = split_message(f"📊 *Анализ новостей*\n\n{analysis}", 4000)

        for i, part in enumerate(parts):
            if i == 0:
                await msg.edit_text(part, parse_mode='Markdown')
            else:
                await update.message.reply_text(part, parse_mode='Markdown')
                await asyncio.sleep(0.5)

    except Exception as e:
        await msg.edit_text(f"❌ Ошибка: {str(e)[:200]}")

application = Application.builder().token(config.TELEGRAM_TOKEN).build()
application.add_handler(CommandHandler("start", start))
application.add_handler(CommandHandler("status", status_command))
application.add_handler(CommandHandler("news", news_command))

print("🚀 Бот запущен! Отправьте /start в Telegram")
print("⚠️ Не закрывайте эту ячейку")

await application.run_polling()